# MD2DS - Master in Data Science and Statistical Learning

**Numerical Calculus and Linear Algebra**

Exercise: SOR

Deadline: 1/5/2024

In [3]:
import numpy as np
from scipy.sparse import csr_matrix, diags, tril
from scipy.sparse.linalg import spsolve_triangular

# Exercise

Consider a linear system $A\mathbf{x} = \mathbf{b}$.
1.  Write a python function `my_SOR(A, x0, b, omega, tol, kmax)` that takes in input

  -  $A\in\mathbb{R}^{m\times m}$  : square matrix
  -  $\mathbf{x}_0\in\mathbb{R}^m$   : a first approximation of the system solution
  -  $\mathbf{b}\in\mathbb{R}^m$   : costant vector
  -  $\omega\in(0,2)$                      : the relxation parameter
  -  $\texttt{tol}\in\mathbb{R}$   : a given tolerance
  -  $\texttt{kmax}\in\mathbb{N}$  : a maximum number of iterations

  and returns
  -  $\mathbf{x}\in\mathbb{R}^m$ : the approximation of the solution of the linear system, obtained by implementing the Successive Over Relaxation method;
  -  $\texttt{it}$ : the number of the computed iterations.

2.  Let
\begin{equation}
      T_n = \pmatrix{ 3       & -1     &        & \huge 0 \\
                     -1       & 3      & \ddots & \\
                              & \ddots & \ddots & -1 \\
                     \huge 0 &      & -1      & 3  }\in\mathbb{R}^{n \times n}
\end{equation}

  and define the sparse (possibly use the sparse format: $\texttt{csr_matrix}$) matrix $A\in\mathbb{R}^{n^2\times n^2}$ as
      \begin{equation}
        A = T_n \otimes I_n + I_n \otimes T_n.
      \end{equation}

  For $n=10,20,\dots,50$ solve the linear system $A\mathbf{x} = \mathbf{b}$ with exact solution $\mathbf{x} = \pmatrix{1\\2\\\vdots\\n^2}\in\mathbb{R}^{n^2}$ using the funcion `my_SOR` implemented at point (1) with
  - $\mathbf{x}_0 = \pmatrix{0\\\vdots\\0}\in\mathbb{R}^{n^2}$
  - $\omega = 0.8, 1, 1.2$
  - $\texttt{toll} = 10^{-5}$
  - $\texttt{kmax} = 10^{5}$
  
  Print
  -  the condition number of $A$,
  -  the number of computed iterations $\texttt{it}$,
  -  the relative error.

Solution 1.1

Considerando che anche nel SOR vale
\begin{equation}
A = M_\omega - N_\omega
\end{equation}
possiamo sviluppare la relazione (seguendo gli stessi passaggi fatti per Gauss-Seidel e Jacobi)
\begin{equation}
\mathbf{x}^{(k+1)} = M_\omega^{-1} N_\omega \mathbf{x}^{(k)} + M_\omega^{-1} \mathbf{b}
\end{equation}
giungendo a
\begin{equation}
\begin{cases}
\mathbf{r}^{(k)} &= \mathbf{b}-A \mathbf{x}^{(k)}\\
M_\omega \mathbf{y}^{(k)}(\omega) &= \mathbf{r}^{(k)} \\
\mathbf{x}^{(k+1)} &= \mathbf{x}^{(k)} + \mathbf{y}^{(k)}
\end{cases}
\end{equation}

Possiamo ulteriolmente scomporre il precedente sistema in base alla definizione di $M_\omega = \omega^{-1} (D - \omega L)$:
\begin{equation}
\omega^{-1}(D - \omega L)\, \mathbf{y}^{(k)} = \mathbf{r}^{(k)} \qquad \Rightarrow \qquad (D-\omega L)\, \mathbf{y}^{(k)}(\omega) = \omega\, \mathbf{r}^{(k)}
\end{equation}

Nella versione di Gauss-Seidel viene calcolato $M\,\mathbf{y}^{(k)} = \mathbf{r}^{(k)}$ sfruttando il meccanismo di risoluzione delle matrici triangolari inferiori  ($M= D-L = \mathtt{tril}(A)$); qui possiamo adattare il procedimento a $D-\omega L$ dove la componente triangolare stretta di A (cioè $-L$) è moltiplicata per $\omega$:
\begin{equation}
A = \begin{bmatrix}
a_{11} & a_{12} & \dots  & a_{1n} \\
a_{21} & a_{22} & \dots  & a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{n1} & a_{n2} & \dots  & a_{nn} \\
\end{bmatrix} \qquad \Rightarrow \qquad
D = \begin{bmatrix}
a_{11} &        &        &      \\
       & a_{22} &        &      \\
       &        & \ddots &      \\
       &        &        &a_{nn}\\
\end{bmatrix}
\quad
L = \begin{bmatrix}
          0 &          &        &            &   \\
   -a_{2,1} &   \ddots &        &            &   \\
     \vdots &   \ddots & \ddots &            &   \\
 -a_{n-1,1} &   \ddots & \ddots &     \ddots &   \\
   -a_{n,1} & -a_{n,2} &  \dots & -a_{n,n-1} & 0 \\
\end{bmatrix}
\end{equation}

\begin{equation}
D-\omega L = \begin{bmatrix}
              a_{1,1} &                  &        &                  &         \\
   \omega\,   a_{2,1} &           \ddots &        &                  &         \\
               \vdots &           \ddots & \ddots &                  &         \\
   \omega\, a_{n-1,1} &           \ddots & \ddots &           \ddots &         \\
   \omega\,   a_{n,1} & \omega\, a_{n,2} &  \dots & \omega\,a_{n,n-1} & a_{n,n} \\
\end{bmatrix}
\end{equation}

Per cui da $\mathtt{tril}(A)\, \mathbf{y} = \mathbf{r}$, ossia
\begin{equation}
\begin{cases}
y_1 = r_1/a_{11}\\
y_2 = (r_2-a_{21} y_1)/a_{22} \\
\vdots\\
y_n = \big(r_n- \sum_{j=1}^{n-1}a_{nj} y_j \big)/ a_{nn}
\end{cases}
\end{equation}

otteniamo per $(D-\omega L)\, \mathbf{y}(\omega) =  \omega\, \mathbf{r}$:

\begin{equation}
\begin{cases}
y_1(\omega) = \omega\, r_1/a_{11}\\
y_2(\omega) = (\omega\, r_2-\omega\, a_{21} y_1(\omega))/a_{22} = \omega\, (r_{2}-a_{21})/a_{22} \\
y_3(\omega) = (\omega r_3 -\omega a_{32} y_2(\omega) - \omega a_{31} y_1(\omega))/a_{33} = \omega (r_3 - a_{32} y_2(\omega) - a_{31} y_1(\omega) ) /a_{33} \\
\vdots\\
y_n(\omega) = \big(\omega\,r_n- \omega \sum_{j=1}^{n-1}a_{nj} y_j(\omega) \big)/ a_{nn} = \omega\, \big(r_n- \sum_{j=1}^{n-1}a_{nj} y_j (\omega) \big)/ a_{nn}
\end{cases}
\end{equation}

Pertanto al passo $k$ avremo che $\mathbf{x}^{(k+1)}$ si ottiene da
\begin{equation}
\begin{cases}
\mathbf{r}^{(k)} &= \mathbf{b}-A \mathbf{x}^{(k)}\\
(D-\omega L)\, \mathbf{y}^{(k)} &= \omega \mathbf{r}^{(k)} \\
\mathbf{x}^{(k+1)} &= \mathbf{x}^{(k)} + \mathbf{y}^{(k)}
\end{cases}
\end{equation}


La funzione $\texttt{lowerTriangularRelaxed}$ risolve il sistema $(D-\omega L) \mathbf{y} = \omega\, \mathbf{r}$, per essere poi integrata nel flusso di elaborazione analogo a quello di Jacobi o Gauss-Seidel.


In [4]:
import numpy as np
from scipy.sparse import csr_matrix, diags, tril
from scipy.sparse.linalg import spsolve_triangular


def lowerTriangularRelaxed(A,r,w):
  # A: lower triangular coefficient matrix
  # b: vector of costant terms
  # w: relax factor
  y = r.copy() # it is necessary to copy b, otherwise it will also be changed as y is.
  y[0] = w*y[0]/A[0,0];
  for i in range(1,A.shape[0]): # row access
    y[i] = w*(y[i] - A[i,0:i]@y[0:i])
    y[i] = y[i] / A[i,i]
  return y

def my_sor(A, x, b, w, toll, kmax):
    n = len(b)
    nrb = np.linalg.norm(b)
    go = True
    it = 1
    while it<=kmax and go:
        r = b - A @ x
        go = (np.linalg.norm(r) > toll*nrb)
        if go:
            it += 1
            x = x + lowerTriangularRelaxed(A,r,w)
    it -= 1
    return x, it



Solution 2



In [5]:
toll = 1e-5
kmax = 1e5

print('n  cond   w    it  rel_err')
for n in range(10, 60, 10):

    T = np.diag(3*np.ones(n)) - np.diag(np.ones(n-1),1) - np.diag(np.ones(n-1),-1)
    A = np.kron(T,np.eye(n))+np.kron(np.eye(n),T)

    N = n**2
    xex = np.ones(N) # exact solution
    for k in range(1,(N+1),1):
        xex[k-1] = k
    b = A @ xex

    x0 = np.zeros(N)
    print('---------------------------')
    for w in (0.8, 1, 1.2):

        x, it = my_sor(A, x0, b, w, toll, kmax)
        xerr = np.linalg.norm(x-xex)/np.linalg.norm(xex)
        print(f'{n} {np.linalg.cond(A):.2f}   {w:.1f}  {it}  {xerr:.2e}')

n  cond   w    it  rel_err
---------------------------
10 4.55   0.8  23  1.09e-05
10 4.55   1.0  15  1.01e-05
10 4.55   1.2  12  1.57e-06
---------------------------
20 4.87   0.8  25  8.08e-06
20 4.87   1.0  16  1.01e-05
20 4.87   1.2  11  4.13e-06
---------------------------
30 4.94   0.8  25  9.48e-06
30 4.94   1.0  17  5.85e-06
30 4.94   1.2  11  4.55e-06
---------------------------
40 4.96   0.8  25  1.02e-05
40 4.96   1.0  17  6.29e-06
40 4.96   1.2  11  4.80e-06
---------------------------
50 4.98   0.8  26  6.74e-06
50 4.98   1.0  17  6.56e-06
50 4.98   1.2  11  4.95e-06


Data la natura tridiagonale di $T_n$, i prodotti $T_n \otimes I_n$ e $T_n \otimes I_n$ producono rispettivamente la matrici
\begin{align*}
T_n \otimes I_n &=
\begin{bmatrix}
3 I_n &   -I_n &        &     \\
 -I_n &   3I_n & \ddots &     \\
      & \ddots & \ddots & -I_n\\
      &        &   -I_n & 3I_n\\
\end{bmatrix}\\
%
I_n \otimes T_n &=
\begin{bmatrix}
  T_n &        &        &     \\
      &    T_n &        &     \\
      &        & \ddots &     \\
      &        &        & 3I_n\\
\end{bmatrix}\\
\end{align*}
da cui
\begin{equation}
T_n \otimes I_n + T_n \otimes I_n =
\begin{bmatrix}
3I_n+T_n &     -I_n &        &         \\
    -I_n & 3I_n+T_n & \ddots &         \\
         &   \ddots & \ddots & -I_n    \\
         &          &   -I_n & 3I_n+T_n\\
\end{bmatrix}
\end{equation}

Per creare la matrice in formato sparso, è stata sfruttata la sua forma a diagonali:
* la diagonale principale ($k=0$) ha tutti elementi 6
* le diagonali per $k=\pm n$ hanno tutti valori -1
* le diagonali per $k=\pm 1$ sono composte tutte da -1, eccetto ogni $n$ elementi dove valgono 0

In [6]:
def CreateMatrixSparseA(n):
    N = n*n
    diag0 = np.full(N,6)
    diagn = np.full(N-n,-1)

    diag1 = np.full(N-1,-1)
    mask = ((1+np.arange(len(diag1))) % n) == 0
    diag1[mask]=0

    Asc = diags([diag0, diag1, diag1, diagn, diagn], [0, -1, 1, -n, n])
    return Asc.tocsr()

def lowerTriangularRelaxedSparse(Asp,r,w):
    d = Asp.diagonal()
    L = tril(Asp, k=-1, format='csr')
    Mw = diags(d,0) + w*L;
    y = spsolve_triangular(Mw, w*r)
    return y

def sparse_sor(Asp, x, b, w, toll, kmax):
    n = len(b)
    nrb = np.linalg.norm(b)
    go = True
    it = 1
    while it<=kmax and go:
        r = b - Asp.dot(x)
        go = (np.linalg.norm(r) > toll*nrb)
        if go:
            it += 1
            x = x + lowerTriangularRelaxedSparse(Asp,r,w)
    it -= 1
    return x, it



#---------------------------------------------------------------
# check sulla correttezza di CreateMatrixSparseA(n)
n = 10
T = np.diag(3*np.ones(n)) - np.diag(np.ones(n-1),1) - np.diag(np.ones(n-1),-1)
Afull = np.kron(T,np.eye(n))+np.kron(np.eye(n),T)
Asparse = CreateMatrixSparseA(n);
print('n=' + str(n) +  ': check |Afull-Asparse| = ' + str(np.linalg.norm(Afull-Asparse.toarray())))



print('\nn  w   it rel_err')
for n in range(10, 60, 10):

    Asp = CreateMatrixSparseA(n)

    N = n**2
    xex = np.ones(N) # exact solution
    for k in range(1,(N+1),1):
        xex[k-1] = k
    b = Asp.dot(xex)

    x0 = np.zeros(N)
    print('-------------------------------------')
    for w in (0.8, 1, 1.2):

        x, it = sparse_sor(Asp, x0, b, w, toll, kmax)
        xerr = np.linalg.norm(x-xex)/np.linalg.norm(xex)
        print(f'{n} {w:.1f} {it} {xerr:.2e}')

n=10: check |Afull-Asparse| = 0.0

n  w   it rel_err
-------------------------------------
10 0.8 23 1.09e-05
10 1.0 15 1.01e-05
10 1.2 12 1.57e-06
-------------------------------------
20 0.8 25 8.08e-06
20 1.0 16 1.01e-05
20 1.2 11 4.13e-06
-------------------------------------
30 0.8 25 9.48e-06
30 1.0 17 5.85e-06
30 1.2 11 4.55e-06
-------------------------------------
40 0.8 25 1.02e-05
40 1.0 17 6.29e-06
40 1.2 11 4.80e-06
-------------------------------------
50 0.8 26 6.74e-06
50 1.0 17 6.56e-06
50 1.2 11 4.95e-06
